In [10]:
import math

def calculatePurchaseReturn(S,R,F,E):
    
    """
    function calculatePurchaseReturn(_supply, _reserveBalance, _reserveRatio,_depositAmount){
    _depositAmount.plus(_reserveBalance)


    R - Reserve-Token Balance (_reserveBalance)
    S - BET Supply            (_supply)
    F - Fixed CRR             (_reseveRatio)
    E - reserve token paid    (_depositAmount)

    T = BET amount received 

    T = S * ((1 + E/R)^F - 1 )
    
    """
    return S * ( math.pow(1.0 + E/R, F) - 1 )
        
def calculateSaleReturn(S,R,F,T):
    """
    R - Reserve-Token Balance (_reserveBalance)
    S - BET Supply            (_supply)
    F - Fixed CRR             (_reseveRatio)
    T = BET paid

    return E = Reserve-Token amount received

    function calculateSaleReturn(uint256 _supply, uint256 _reserveBalance, uint16 _reserveRatio, uint256 _sellAmount) public constant returns (uint256 amount) {

    """
    return R * ( math.pow((1.0+T/S)  ,(1.0/F)) -1 )


In [11]:
# These functions mimic the EVM-implementation

PRECISION = 32;  # fractional bits

def uint128(val):
    val = int(val)
    return int(val & 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF)

def uint256(val):
    val = int(val)
    return int(val & 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF)
    
def ln(_numerator, _denominator):
    return fixedLoge(_numerator << PRECISION) - fixedLoge(_denominator << PRECISION);

def fixedLoge(_x) :
    x = uint256(_x)
    return math.floor((fixedLog2(_x) * 1488522236) >> 31); # 1,488,522,236 = ln(2) * (2 ^ 31)
    

def fixedLog2( _x) :
    _x = uint256(_x)
    fixedOne = uint256(1 << PRECISION);
    fixedTwo = uint256(2 << PRECISION);

    lo = 0;
    hi = 0;

    while _x < fixedOne:
        _x <<= 1;
        lo += fixedOne;
    

    while _x >= fixedTwo:
        _x >>= 1;
        hi += fixedOne;
    

    for i in range(0, PRECISION,1):
        _x = (_x * _x) >> PRECISION;
        if (_x >= fixedTwo) :
            _x >>= 1;
            hi += uint256(1 << (PRECISION - 1 - i));

    return math.floor(hi - lo);

def fixedExp( _x) :
    _x = uint256(_x)
    fixedOne = uint256(1 << PRECISION);

    # TODO: change to constant array instead of calculating each time        
    #uint256[34 + 1] memory ni;
    ni = []
    ni.append( 295232799039604140847618609643520000000)
    for n in range( 1,  35,1 ) :
        ni.append(math.floor(ni[n - 1] / n))
        #ni[n] = ;

    res = uint256(ni[0] << PRECISION);
    xi = uint256(fixedOne);
    for i in range(1, 35,1 ):
        xi = uint256((xi * _x) >> PRECISION);
        res += math.floor(xi * ni[i]);
        res = uint256(res)
        
    return math.floor(res / ni[0]);

def power(_baseN,_baseD, _expN, _expD):
    return (fixedExp(ln(_baseN, _baseD) * _expN / _expD), 1 << PRECISION);


def calculatePurchaseReturnSolidity(S,R,F,E):
    """The 'solidity' version, which matches pretty closely to what 
    happens under the EVM hood"""

    _supply = int(S)
    _reserveBalance = int(R)
    _reserveRatio = int(F)
    _depositAmount = int(E)

    uint128_1 = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
    
    if _supply > uint128_1 or _reserveBalance > uint128_1 or _depositAmount > uint128_1:
        raise Exception("Out of bounds")

    # (E+R)^R
    (resN, resD) = power(uint128(_depositAmount + _reserveBalance), uint128(_reserveBalance), _reserveRatio, 100);
    
    return uint256((_supply * resN / resD) - _supply)

        
def calculateSaleReturnSolidity(S, R, F,  T):
    """The 'solidity' version, which matches pretty closely to what 
    happens under the EVM hood"""
    _supply = int(S)
    _reserveBalance = int(R)
    _reserveRatio = int(F)
    _sellAmount = int(T)
    
    uint128_1 = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
    
    if (_supply == 0 or _reserveBalance == 0 or _reserveRatio < 1 or _reserveRatio > 99 or  _sellAmount == 0):# validate input
            raise Exception("Error %s %s %s" % (_supply, _reserveBalance,_reserveRatio))
        # limiting input to 128bit to provide *some* overflow protection while keeping the interface generic 256bit
        # TODO: will need to revisit this
    if (_supply > uint128_1 or _reserveBalance > uint128_1 or _sellAmount > uint128_1):
        raise Exception("Out of bounds")

    (resN, resD) = power(uint128(_sellAmount + _supply), uint128(_supply), 100, _reserveRatio);
    resN = uint256(resN)
    resD = uint256(resD)
    return uint256((_reserveBalance * resN / resD) - _reserveBalance);



In [12]:
class exactCrowdsale():
    
    def __init__(self):
        self.R = 63000.0# 63000 #
        self.S = 300000.0
        self.F = .21 # 21% CRR , 
        
    def __str__(self):
        
        unit_price = self.R / (self.S*self.F)
        return "\n" \
        +" ETH Reserve      %f\n" % self.R \
        +" BGT Supply       %f\n" % self.S \
        +" BGT Market-cap:  %f\n" % (self.S * unit_price)\
        + " BGT Unit price   %f\n" % unit_price 

    def buyWithReserveToken(self, E):
        "This is the exact formula"
        T = calculatePurchaseReturn(self.S,self.R,self.F,E)
        self.R += E
        self.S += T
        return T

    def sellForReserveToken(self, T):
        unit_price = self.R / (self.S*self.F)
        E = calculateSaleReturn(self.S,self.R,self.F,T)
        self.R -= E
        self.S -= T
        return E

class exampleCrowdsale(exactCrowdsale):
    
    def __init__(self):
        self.R = 63000.0# 63000 #
        self.S = 300000.0
        self.F = .21 # 21% CRR , 
        
    def __str__(self):
        
        unit_price = self.R / (self.S*self.F)
        return "\n" \
        +" ETH Reserve      %f\n" % self.R \
        +" BGT Supply       %f\n" % self.S \
        +" BGT Market-cap:  %f\n" % (self.S * unit_price)\
        + " BGT Unit price   %f\n" % unit_price 
        
    def buyWithReserveToken(self, E):
        
        unit_price = self.R / (self.S*self.F)
        
        T_correct = calculatePurchaseReturn(self.S,self.R,self.F,E)
        T = calculatePurchaseReturn2(self.S,self.R,self.F*100,E)

        #print("ETH received     %f " % E)
        print("BGT Issued       %f (%f, diff %f %%)" % (T, T_correct, (T-T_correct)*100/T))
        
        conversion_rate = E/T
        T = math.floor(T)
        
        self.R += E
        self.S += T
        unit_price_new = self.R / (self.S*self.F)
        return T

    def sellForReserveToken(self, T):

        unit_price = self.R / (self.S*self.F)

        E = calculateSaleReturn2(self.S,self.R,self.F*100,T)
        E_correct = calculateSaleReturn(self.S,self.R,self.F,T)
        
        #E = math.floor(E)
        self.R -= E
        self.S -= T

        unit_price_new = self.R / (self.S*self.F)

        conversion_rate = E/T
        print("ETH issued     %f (%f, diff %f %%)" % (E, E_correct,(E-E_correct)*100/E))       
        print("BGT received       %f" % -T)
        print("Conversion rate  %f" % conversion_rate)
        
        return E




In [13]:
S = 300000.0
R = 63000.0
F= .21

print("var p = [")
for i in range(1, 1000,2):
    E = float(i * i) # Goes up to 1 million ether 
    T = calculatePurchaseReturn(S,R,F,E)
    print("\t[%d,%d,%d,%d,%d, %f]," % ( int(S), int(R), int(100*F), int(E),math.floor(T), T ))
print("];")
print("module.exports.purchaseReturns = p;")

print("var s = [")
for i in range(1, 1000,2):
    T = float(i * i) # Goes up to 1 million tokens 
    E = calculateSaleReturn(S,R,F,T)
    print("\t[%d,%d,%d,%d,%d, %f]," % ( int(S), int(R), int(100*F), int(T),math.floor(E), E ))
print("];")
print("module.exports.saleReturns = s;")

var p = [
	[300000,63000,21,1,0, 0.999994],
	[300000,63000,21,9,8, 8.999492],
	[300000,63000,21,25,24, 24.996082],
	[300000,63000,21,49,48, 48.984953],
	[300000,63000,21,81,80, 80.958895],
	[300000,63000,21,121,120, 120.908308],
	[300000,63000,21,169,168, 168.821213],
	[300000,63000,21,225,224, 224.683264],
	[300000,63000,21,289,288, 288.477765],
	[300000,63000,21,361,360, 360.185691],
	[300000,63000,21,441,439, 439.785703],
	[300000,63000,21,529,527, 527.254181],
	[300000,63000,21,625,622, 622.565241],
	[300000,63000,21,729,725, 725.690771],
	[300000,63000,21,841,836, 836.600456],
	[300000,63000,21,961,955, 955.261816],
	[300000,63000,21,1089,1081, 1081.640240],
	[300000,63000,21,1225,1215, 1215.699019],
	[300000,63000,21,1369,1357, 1357.399392],
	[300000,63000,21,1521,1506, 1506.700583],
	[300000,63000,21,1681,1663, 1663.559848],
	[300000,63000,21,1849,1827, 1827.932512],
	[300000,63000,21,2025,1999, 1999.772026],
	[300000,63000,21,2209,2179, 2179.030006],
	[300000,63000,21,2401,2365

In [14]:
# Random tests

import random, math


print("module.exports.randomPurchaseReturns = [")
for i in range(1, 100):
    S = float(random.randint(1e6, 3e6))
    F = random.randint(1, 100 )
    R = math.floor(F*S / 100)
    E = float(random.randint(700, 300000))
    T = calculatePurchaseReturn(S,R,float(F)/100,E)
    print("\t[%d,%d,%d,%d,%d, %f]," % ( int(S), int(R), int(F), int(E),math.floor(T), T ))
print("];")



print("module.exports.randomSaleReturns = [")
for i in range(1, 100):
    S = float(random.randint(1e6, 3e6))
    F = random.randint(1, 100 )
    R = math.floor(F*S / 100)
    T = float(random.randint(700, 300000))
    E = calculateSaleReturn(S,R,float(F)/100,T)
    print("\t[%d,%d,%d,%d,%d, %f]," % ( int(S), int(R), int(F), int(T),math.floor(E), E ))
print("];")



module.exports.randomPurchaseReturns = [
	[2916997,729249,25,107124,101682, 101682.621987],
	[1641873,1100054,67,53215,52799, 52799.149773],
	[1232497,838097,68,44676,44303, 44303.677918],
	[2238835,2171669,97,279464,278947, 278947.063552],
	[1558129,1121852,72,175566,171955, 171955.616401],
	[1919146,498977,26,257241,219088, 219088.343444],
	[2420537,484107,20,21570,21195, 21195.553292],
	[1667445,466884,28,5143,5122, 5122.739300],
	[2562373,2280511,89,252668,251188, 251188.090098],
	[1653911,1372746,83,142143,140939, 140939.806598],
	[2702778,2513583,93,149920,149613, 149613.527865],
	[1183525,284046,24,167047,138951, 138951.655349],
	[2819807,930536,33,95463,92357, 92357.641042],
	[2138570,1218984,57,172170,167266, 167266.323384],
	[2289975,1740381,76,262070,257606, 257606.520594],
	[1705693,596992,35,223650,200933, 200933.817317],
	[2572154,514430,20,249857,211934, 211934.293148],
	[2005836,1704960,85,66814,66620, 66620.539625],
	[1718851,1718851,100,115289,115289, 115289.000000],


In [15]:
[S,R,F,T] = [2501391,725403,28,291459]
E = calculateSaleReturnSolidity(S,R,F,T)
E_correct = calculateSaleReturnExact(S,R,F/100,T)
print(E)
print(E_correct)

349896
349896.50917568104
